In [8]:
library(PharmacoGx, quietly=TRUE)
library(Biobase, quietly=TRUE)
library(calibrate, quietly=TRUE)
library(forestplot, quietly=TRUE)
library(survcomp, quietly=TRUE)
library(DESeq2, quietly=TRUE)
library(dplyr, quietly=TRUE)
library(stringr, quietly=TRUE)
library(gimme, quietly=TRUE)

In [6]:
options(stringsAsFactors = F)
#match cell.id to unique.cellid from PharmacoGx
matchToIDTable <- function(ids,tbl, column, returnColumn="unique.cellid") {
  sapply(ids, function(x) {
    myx <- grep(paste0("((///)|^)",Hmisc::escapeRegex(x),"((///)|$)"), tbl[,column])
    if(length(myx) > 1){
      stop("Something went wrong in curating ids, we have multiple matches")
    }
    if(length(myx) == 0){return(NA_character_)}
    return(tbl[myx, returnColumn])
  })
}

In [7]:
#creates list of all gene-drug combinations
expand.grid.unique <- function(x, y, include.equals=FALSE)
{
  x <- unique(x)
  y <- unique(y)
  g <- function(i)
  {
    z <- setdiff(y, x[seq_len(i-include.equals)])
    
    if(length(z)) cbind(x[i], z, deparse.level=0)
  }
  do.call(rbind, lapply(seq_along(x), g))
}

## Read in cell line annotations & convert to unique.cellid (gCSI, CCLE, GDSC)

In [9]:
##read in current cell annotations
cell_all <- read.csv(file = "rnaseq_meta/cell_annotation_all.csv", na.strings=c("", " ", "NA"))

In [10]:
#read in gcsi cell annotations
gcsi <- read.csv(file = "rnaseq_meta/gcsi_rnaseq_meta.csv")
gcsi$cellid <- matchToIDTable(ids=gcsi$Cell_line , tbl = cell_all, column = "GNE.cellid", returnColumn = "unique.cellid")
rownames(gcsi) <- gcsi$alias

#read in ccle cell annotations
ccle <- read.csv(file = "rnaseq_meta/ccle_rnaseq_meta.csv")
ccle$cellid <- matchToIDTable(ids=ccle$Cell_Line , tbl = cell_all, column = "CCLE.cellid", returnColumn = "unique.cellid")
rownames(ccle) <- ccle$Run

#read in gdsc cell annotations
gdsc <- read.csv(file = "rnaseq_meta/gdsc_rnaseq_meta.txt", sep = "\t")
gdsc <- gdsc[which(!gdsc$Comment.SUBMITTED_FILE_NAME. == "15552_5.cram"),]
gdsc$cellid <- matchToIDTable(ids=gdsc$Source.Name, tbl = cell_all, column = "GDSC_rnaseq.cellid", returnColumn = "unique.cellid")
gdsc$files <- gsub(".cram","",gdsc$Comment.SUBMITTED_FILE_NAME.)
rownames(gdsc) <- gdsc$files

## Create gene count matrix (gCSI, CCLE, GDSC) for each tool (CIRI2, CIRCexplorer2)

In [11]:
#load Gencode v33lift37 (GRCh37) gene/transcript annotations used for circRNA detection in both tools
load("Gencode.v33lift37.annotation.RData")

In [12]:
#create gene matrix for CIRI2 results
summarizeCIRIMatrix <- function(dir_path){
  
  ciri_files <- list.files(path=dir_path,  pattern = "\\.tsv$", full.names = T)
  gene_matrix <- data.frame(matrix(ncol=length(ciri_files), nrow = length(rownames(features_gene))))
  rownames(gene_matrix) <- rownames(features_gene)
  
  for (f in 1:length(ciri_files)) {
    sample <- read.table(file = ciri_files[f], 
                         sep = '\t', 
                         skip = 1,
                         header = FALSE)
    
    colnames(sample) <- c("circRNA_ID","chr","circRNA_start","circRNA_end","junction_reads", "SM_MS_SMS", "non_junction_reads", "junction_reads_ratio", "circRNA_type", "gene_id", "strand","junction_reads_ID")
    sample_name <- gsub("\\..*","", ciri_files[f])
    sample_name <- gsub(".*/","", sample_name)
    sample <- sample[which(sample$circRNA_type=="exon"),]
    sample <- sample[which(sample$junction_reads >= 2),]
    
    gene_reads <- sample %>% 
      group_by(gene_id) %>% 
      summarise(junction_reads = sum(junction_reads))
    
    gene_reads <- as.data.frame(gene_reads)
    gene_reads <- gene_reads[str_count(gene_reads$gene_id, ',')  <= 1,]
    gene_reads$gene_id <- gsub(",", "", gene_reads$gene_id)
    rownames(gene_reads) <- gene_reads$gene_id
    gene_matrix[rownames(gene_reads), f] <- gene_reads$junction_reads
    names(gene_matrix)[f] <- sample_name
    
  }
  gene_matrix <- gene_matrix[rowSums(is.na(gene_matrix)) != ncol(gene_matrix), ]#remove rows that have NO exp (NA) for any sample
  gene_matrix[is.na(gene_matrix)] <- 0
  return(gene_matrix)
} 

In [13]:
gcsi_ciri_matrix <- summarizeCIRIMatrix(dir_path = "results/CIRI2/gCSI/result")
ccle_ciri_matrix <- summarizeCIRIMatrix(dir_path = "results/CIRI2/CCLE/result")
gdsc_ciri_matrix <- summarizeCIRIMatrix(dir_path = "results/CIRI2/GDSC/result")


#Because there are genes with no circRNA exp (0), we need to log2 normalize + 1 for each gene
gcsi_ciri_matrix_norm <- log2(gcsi_ciri_matrix + 1)
ccle_ciri_matrix_norm <- log2(ccle_ciri_matrix + 1)
gdsc_ciri_matrix_norm <- log2(gdsc_ciri_matrix + 1)

#use unique.cellids for column names of gene matrix & keep same column name order as gCSI
colnames(gcsi_ciri_matrix_norm) <- gsub("gcsi","", colnames(gcsi_ciri_matrix_norm))
colnames(gcsi_ciri_matrix_norm) <- gcsi$cellid[match(colnames(gcsi_ciri_matrix_norm), rownames(gcsi))]

colnames(gdsc_ciri_matrix_norm) <- gsub("gdsc","", colnames(gdsc_ciri_matrix_norm))
colnames(gdsc_ciri_matrix_norm) <- gdsc$cellid[match(colnames(gdsc_ciri_matrix_norm), rownames(gdsc))]
gdsc_ciri_matrix_norm <- gdsc_ciri_matrix_norm[names(gcsi_ciri_matrix_norm)]

colnames(ccle_ciri_matrix_norm) <- ccle$cellid[match(colnames(ccle_ciri_matrix_norm), rownames(ccle))]
ccle_ciri_matrix_norm <- ccle_ciri_matrix_norm[names(gcsi_ciri_matrix_norm)]

In [14]:
#create gene matrix for CIRCexplorer2 results
summarizeCIRCMatrix <- function(dir_path){
  
  circ_files <- list.files(path=dir_path,  pattern = ".txt", full.names = T, recursive = TRUE)
  gene_matrix <- data.frame(matrix(ncol=length(circ_files), nrow = length(rownames(features_gene))))
  rownames(gene_matrix) <- rownames(features_gene)
  
  for (f in 1:length(circ_files)) {
    sample <- read.table(file = circ_files[f], 
                         sep = '\t', 
                         header = FALSE)
    
    colnames(sample) <- c("chrom","start","end", "name","score","strand","thickStart","thickEnd","itemRgb","exonCount","exonSizes", "exonOffsets", "readNumber", "circType", "geneName", "isoformName", "index", "flankIntron")
    sample_name <- sub(".*/ *(.*?) *.txt.*", "\\1", circ_files[f])
    sample_name <- gsub("_circularRNA_known", "", sample_name)
    sample <- sample[which(sample$circType=="circRNA"),]
    sample <- sample[which(sample$readNumber >= 2),]
    
    #get gene-id from gencode v33lift37 feature_transcript data-frame, as circexplorer2 does not provide it in result files
    
    sample$gene_id <- features_transcript$gene_id[match(sample$isoformName, features_transcript$transcript_id)]
    
    #group sum of read number per gene_id
    gene_reads <- sample %>% 
      group_by(gene_id) %>% 
      summarise(readNumber = sum(readNumber))
    
    gene_reads <- as.data.frame(gene_reads)
    
    rownames(gene_reads) <- gene_reads$gene_id
    gene_matrix[rownames(gene_reads), f] <- gene_reads$readNumber
    names(gene_matrix)[f] <- sample_name
    
  }
  gene_matrix <- gene_matrix[rowSums(is.na(gene_matrix)) != ncol(gene_matrix), ]#remove rows that have NO exp (NA) for any sample
  gene_matrix[is.na(gene_matrix)] <- 0
  return(gene_matrix)
} 

In [15]:
gcsi_circ_matrix <- summarizeCIRCMatrix(dir_path = "results/CIRCexplorer2/gCSI/unmapped_method/annotate")
ccle_circ_matrix <- summarizeCIRCMatrix(dir_path = "results/CIRCexplorer2/CCLE/unmapped_method/annotate")
gdsc_circ_matrix <- summarizeCIRCMatrix(dir_path = "results/CIRCexplorer2/GDSC/unmapped_method/annotate")


#Because there are genes with no circRNA exp (0), we need to log2 normalize + 1 for each gene
gcsi_circ_matrix_norm <- log2(gcsi_circ_matrix + 1)
ccle_circ_matrix_norm <- log2(ccle_circ_matrix + 1)
gdsc_circ_matrix_norm <- log2(gdsc_circ_matrix + 1)


#use unique.cellids for column names of gene matrix & keep same column name order as gCSI
colnames(gcsi_circ_matrix_norm) <- gsub("gcsi","", colnames(gcsi_circ_matrix_norm))
colnames(gcsi_circ_matrix_norm) <- gcsi$cellid[match(colnames(gcsi_circ_matrix_norm), rownames(gcsi))]

colnames(gdsc_circ_matrix_norm) <- gsub("gdsc","", colnames(gdsc_circ_matrix_norm))
colnames(gdsc_circ_matrix_norm) <- gdsc$cellid[match(colnames(gdsc_circ_matrix_norm), rownames(gdsc))]
gdsc_circ_matrix_norm <- gdsc_circ_matrix_norm[names(gcsi_circ_matrix_norm)]

colnames(ccle_circ_matrix_norm) <- ccle$cellid[match(colnames(ccle_circ_matrix_norm), rownames(ccle))]
ccle_circ_matrix_norm <- ccle_circ_matrix_norm[names(gcsi_circ_matrix_norm)]


## Compute concordance index (CI) for all datasets (gCSI, CCLE, GDSC)

### import PSets (drug sensitivity data)

In [2]:
gCSI <- readRDS("gCSI.rds") #2017 drug sensitivity

In [3]:
CTRPv2 <- readRDS("CTRPv2.rds")

In [1]:
CCLE <- readRDS("CCLE.rds")

In [4]:
GDSC2 <- readRDS("GDSC2.rds") #v8.2 (Feb2020 drug sensitivity)

### get average circRNA count for SR technical replicate in gCSI

In [17]:
#CIRI2 matrix
combined_mean <- as.data.frame(rowMeans(gcsi_ciri_matrix_norm[, c("SR", "SR")], na.rm = TRUE))
idx <- which(duplicated(colnames(gcsi_ciri_matrix_norm)))
gcsi_ciri_matrix_norm <- gcsi_ciri_matrix_norm[,-idx]
v <- combined_mean$`rowMeans(gcsi_ciri_matrix_norm[, c("SR", "SR")], na.rm = TRUE)`
gcsi_ciri_matrix_norm$SR <- v

In [18]:
#CIRCexplorer2 matrix
combined_mean <- as.data.frame(rowMeans(gcsi_circ_matrix_norm[, c("SR", "SR")], na.rm = TRUE))
idx <- which(duplicated(colnames(gcsi_circ_matrix_norm)))
gcsi_circ_matrix_norm <- gcsi_circ_matrix_norm[,-idx]
v <- combined_mean$`rowMeans(gcsi_circ_matrix_norm[, c("SR", "SR")], na.rm = TRUE)`
gcsi_circ_matrix_norm$SR <- v

### summarizeDrugSensitivity for all datasets

In [53]:
intersected_cells <- colnames(gcsi_ciri_matrix_norm) #intersected biological replicates
gcsi_summarize <- summarizeSensitivityProfiles(pSet = gCSI, sensitivity.measure = "aac_recomputed", fill.missing = F)
ctrpv2_summarize <- summarizeSensitivityProfiles(pSet = CTRPv2, sensitivity.measure = "aac_recomputed", cell.lines = intersected_cells, fill.missing = F)
ccle_summarize <- summarizeSensitivityProfiles(pSet = CCLE, sensitivity.measure = "aac_recomputed", cell.lines = intersected_cells, fill.missing = F)
gdsc_summarize <- summarizeSensitivityProfiles(pSet = GDSC2, sensitivity.measure = "aac_recomputed", cell.lines = intersected_cells, fill.missing = F)


### compute CI for each gene-drug combination

In [41]:
#computes concordance index (CI) for each dataset, using Gencode v33lift37 annotations
#circRNA_normalzed_data = log2 + 1 circRNA matrix (CIRI2, CIRCexplorer2)
#sensitivity_data = result of 'summarizeSensitivityProfiles' in PharmacoGx
#samples = commonSamples

computeCI <- function(circRNA_normalized_data, sensitivity_data, samples){
  
  genes <- rownames(circRNA_normalized_data)
  drugs <- unique(rownames(sensitivity_data))
  commonSamples <- samples
  combinations <- as.data.frame(expand.grid.unique(genes, drugs, include.equals = TRUE))
  combinations$ci <- NA
  combinations$pvalue <- NA
  colnames(combinations) <- c("gene","drug","ci","pvalue")
  
  for (i in 1:nrow(combinations)){
    #print(paste0(i, " out of ", nrow(combinations), " complete"))
    tt <- sensitivity_data[combinations[,2][i],commonSamples]
    tt[which(is.na(tt))] <- 0 #some sensitivities are NA due to filterNoisyCurve function, which causes error when running CI with survcomp
    ci <- survcomp::concordance.index(tt, surv.time = unlist(-gcsi_ciri_matrix_norm[combinations[,1][i], commonSamples]), surv.event = rep(1,length(sensitivity_data[commonSamples])),outx = F, method="noether")
    combinations$pvalue[i] <- ci$p.value
    combinations$ci[i] <- ci$c.index
  }
  
  return(combinations)
}

#### gCSI

In [39]:
#gCSI compute CI
commonSamples <- intersect(colnames(gcsi_summarize),colnames(gcsi_ciri_matrix_norm)) #only samples with drug sensitivity in gCSI
gcsi_ci_result <- computeCI(circRNA_normalized_data=gcsi_ciri_matrix_norm, sensitivity_data=gcsi_summarize, samples = commonSamples)

[1] "1 out of 29712 complete"
[1] "2 out of 29712 complete"
[1] "3 out of 29712 complete"
[1] "4 out of 29712 complete"
[1] "5 out of 29712 complete"
[1] "6 out of 29712 complete"
[1] "7 out of 29712 complete"
[1] "8 out of 29712 complete"
[1] "9 out of 29712 complete"
[1] "10 out of 29712 complete"
[1] "11 out of 29712 complete"
[1] "12 out of 29712 complete"
[1] "13 out of 29712 complete"
[1] "14 out of 29712 complete"
[1] "15 out of 29712 complete"
[1] "16 out of 29712 complete"
[1] "17 out of 29712 complete"
[1] "18 out of 29712 complete"
[1] "19 out of 29712 complete"
[1] "20 out of 29712 complete"
[1] "21 out of 29712 complete"
[1] "22 out of 29712 complete"
[1] "23 out of 29712 complete"
[1] "24 out of 29712 complete"
[1] "25 out of 29712 complete"
[1] "26 out of 29712 complete"
[1] "27 out of 29712 complete"
[1] "28 out of 29712 complete"
[1] "29 out of 29712 complete"
[1] "30 out of 29712 complete"
[1] "31 out of 29712 complete"
[1] "32 out of 29712 complete"
[1] "33 out of 29

In [58]:
#CCLE compute CI (circRNA samples not found in gCSI sensitivity)
commonSamples <- intersect(colnames(ccle_summarize),colnames(gcsi_ciri_matrix_norm)) #only samples with drug sensitivity in CCLE
ccle_ci_result <- computeCI(circRNA_normalized_data=gcsi_ciri_matrix_norm, sensitivity_data=ccle_summarize, samples = commonSamples)

In [60]:
#GDSC compute CI (circRNA samples not found in gCSI sensitivity)
commonSamples <- intersect(colnames(gdsc_summarize),colnames(gcsi_ciri_matrix_norm)) #only samples with drug sensitivity in GDSC
gdsc_ci_result <- computeCI(circRNA_normalized_data=gcsi_ciri_matrix_norm, sensitivity_data=gdsc_summarize, samples = commonSamples)

In [64]:
#CTRPv2 compute CI (circRNA samples not found in gCSI sensitivity)
commonSamples <- intersect(colnames(ctrpv2_summarize),colnames(gcsi_ciri_matrix_norm)) #only samples with drug sensitivity in GDSC
ctrpv2_ci_result <- computeCI(circRNA_normalized_data=gcsi_ciri_matrix_norm, sensitivity_data=ctrpv2_summarize, samples = commonSamples)

In [65]:
#save all computed CI's
save(gcsi_ci_result, file="gcsi_ci_result/gcsi_CI.RData")
save(ccle_ci_result, file="gcsi_ci_result/ccle_CI.RData")
save(gdsc_ci_result, file="gcsi_ci_result/gdsc_CI.RData")
save(ctrpv2_ci_result, file="/gcsi_ci_result/ctrpv2_CI.RData")

In [62]:
nrow(gdsc_ci_result)

[1] 352830

#### CCLE